# TP1: Parcours de protons dans la matière
## PHY-3500: Physique numérique (H2021) - 19 février 2021


Mélissa Cinq-Mars (111 179 946)

Pierre-Olivier Janvier (111 187 987)

Maxence Larose (111 188 447)